In [4]:
import csv
from datetime import datetime
import time
import random
import pickle

import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.stats import weightedtau, kendalltau
from scipy.stats import norm
from scipy.linalg import null_space
from sklearn.preprocessing import normalize
import pandas as pd

from pprint import pprint
from copy import deepcopy
from collections import defaultdict

import trueskill

#### After run `preprocess_for_downstream.py`

current_dir = "../dataset/Preprocess/"

In [5]:
d = pd.read_csv("paper_info.txt")
d

,paperid,venue,year
0,1,Linux Journal,1998
1,2,Integration the VLSI Journal,1984
2,3,International Journal of Parallel Programming,1984
3,4,International Journal of Parallel Programming,1984
4,5,International Journal of Parallel Programming,1984
...,...,...,...
2092140,2092352,ACM Transactions on Software Engineering and M...,2008
2092141,2092353,ACM Transactions on Software Engineering and M...,2008
2092142,2092354,ACM Transactions on Software Engineering and M...,2008
2092143,2092355,XRDS: Crossroads The ACM Magazine for Student...,2010


In [34]:
venue_set = set()
for i, row in d.iterrows():
    venue_set.add(row["venue"])
print(len(venue_set))

263816


In [76]:
venue_category = {
    "Computational Linguistics": ["Meeting of the Association for Computational Linguistics",
                                 "ACL",
                                 "Conference on Empirical Methods in Natural Language Processing",
                                 "EMNLP",
                                 "Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies",
                                 "HLT-NAACL",
                                 "Conference of the European Chapter of the Association for Computational Linguistics",
                                 "EACL",
                                 "Transactions of the Association for Computational Linguistics",
                                 "International Conference on Computational Linguistics",
                                 "COLING",
                                 "Workshop on Machine Translation",
                                 "International Conference on Language Resources and Evaluation",
                                 "LREC",
                                 "International Workshop on Semantic Evaluation",
                                 "Conference on Computational Natural Language Learning",
                                 "CoNLL",
                                 "Computer Speech & Language",
                                 "International Joint Conference on Natural Language Processing",
                                 "IJCNLP",
                                 "Computational Linguistics",
                                 "IEEE Spoken Language Technology Workshop",
                                 "SLT",
                                 "Annual Meeting of the Special Interest Group on Discourse and Dialogue",
                                 "SIGDIAL",
                                 "International Conference on Natural Language Generation",
                                 "Workshop on Representation Learning for NLP",
                                 "Language Resources and Evaluation",
                                 "Natural Language Engineering",
                                 "Workshop on Innovative Use of NLP for Building Educational Applications"],
    
    "Computer Graphics": ["ACM Transactions on Graphics",
                          "TOG",
                          "IEEE Transactions on Visualization and Computer Graphics",
                          "Computer Graphics Forum",
                          "The Visual Computer",
                          "Computers & Graphics",
                          "IEEE Computer Graphics and Applications",
                          "ACM Symposium on Virtual Reality Software and Technology",
                          "Computer Aided Geometric Design",
                          "Conference on Graphics, Patterns and Images",
                          "Visual Informatics",
                          "IEEE Pacific Visualization Symposium",
                          "Information Visualization",
                          "Computer Animation and Virtual Worlds",
                          "SIGGRAPH Asia Posters",
                          "Proceedings of the ACM on Computer Graphics and Interactive Techniques",
                          "IEEE Symposium on Visual Analytics Science and Technology",
                          "International Conference on Information Visualization",
                          "IEEE Visualization Conference",
                          "Symposium on Graph Drawing",
                          "GD",
                          "Graphics Interface Conference"
                         ],
    
    "Computer Networks & Wireless Communication": ["IEEE Communications Surveys & Tutorials",
                                                   "IEEE Communications Magazine",
                                                   "IEEE Transactions on Vehicular Technology",
                                                   "IEEE Transactions on Wireless Communications",
                                                   "IEEE Journal on Selected Areas in Communications",
                                                   "IEEE Transactions on Communications",
                                                   "IEEE Wireless Communications",
                                                   "Journal of Network and Computer Applications",
                                                   "IEEE Network",
                                                   "IEEE Communications Letters",
                                                   "IEEE Transactions on Mobile Computing",
                                                   "Computer networks",
                                                   "IEEE Wireless Communications Letters",
                                                   "IEEE International Conference on Communications",
                                                   "IEEE Systems Journal",
                                                   "IEEE International Conference on Computer Communications",
                                                   "INFOCOM",
                                                   "IEEE International Conference on Computer and Communications",
                                                   "Computer Communications",
                                                   "IEEE/ACM Transactions on Networking",
                                                   "IEEE GLOBECOM"
                                                  ],
    
    "Computer Vision & Pattern Recognition": ["IEEE/CVF Conference on Computer Vision and Pattern Recognition",
                                              "IEEE/CVF International Conference on Computer Vision",
                                              "European Conference on Computer Vision",
                                              "IEEE Transactions on Pattern Analysis and Machine Intelligence",
                                              "IEEE Transactions on Image Processing",
                                              "Pattern Recognition",
                                              "IEEE/CVF Computer Society Conference on Computer Vision and Pattern Recognition Workshops",
                                              "CVPRW",
                                              "Medical Image Analysis",
                                              "IEEE/CVF Winter Conference on Applications of Computer Vision",
                                              "WACV",
                                              "International Journal of Computer Vision",
                                              "British Machine Vision Conference",
                                              "BMVC",
                                              "Pattern Recognition Letters",
                                              "IEEE/CVF International Conference on Computer Vision Workshops",
                                              "ICCVW",
                                              "IEEE International Conference on Image Processing",
                                              "ICIP",
                                              "Computer Vision and Image Understanding",
                                              "IEEE International Conference on Automatic Face & Gesture Recognition",
                                              "International Conference on Document Analysis and Recognition",
                                              "Journal of Visual Communication and Image Representation",
                                              "Asian Conference on Computer Vision",
                                              "ACCV",
                                              "Image and Vision Computing"
                                             ],
    
    "Computing Systems": ["IEEE Internet of Things Journal"
                         "Future Generation Computer Systems",
                          "USENIX Symposium on Networked Systems Design and Implementation",
                          "IEEE Transactions on Services Computing",
                          "The Journal of Supercomputing",
                          "IEEE Transactions on Parallel and Distributed Systems",
                          "	International Conference on Architectural Support for Programming Languages and Operating Systems",
                          "ASPLOS",
                          "Cluster Computing",
                          "USENIX Annual Technical Conference",
                          "International Symposium on Computer Architecture",
                          "ISCA",
                          "Journal of Parallel and Distributed Computing",
                          "IEEE/ACM International Symposium on Microarchitecture",
                          "IEEE Transactions on Cloud Computing",
                          "IEEE International Symposium on High Performance Computer Architecture",
                          "International Conference on Distributed Computing Systems",
                          "ICDCS",
                          "ACM European Conference on Computer Systems",
                          "Concurrency and Computation: Practice and Experience",
                          "International Conference for High Performance Computing, Networking, Storage and Analysis",
                          "Symposium on Operating Systems Principles",
                          "	Internet of Things"
                         ],
    
    "Databases & Information Systems": ["International World Wide Web Conferences",
                                        "WWW",
                                        "IEEE Transactions on Knowledge and Data Engineering",
                                        "ACM SIGIR Conference on Research and Development in Information Retrieval",
                                        "International Conference on Very Large Databases",
                                        "Information Processing & Management",
                                        "ACM International Conference on Web Search and Data Mining",
                                        "ACM International Conference on Information and Knowledge Management",
                                        "ACM SIGMOD International Conference on Management of Data",
                                        "Journal of Big Data",
                                        "International Conference on Data Engineering",
                                        "International Conference on Web and Social Media",
                                        "ICWSM",
                                        "IEEE International Conference on Big Data",
                                        "Knowledge and Information Systems",
                                        "ACM Conference on Recommender Systems",
                                        "IEEE Transactions on Big Data",
                                        "Information Systems",
                                        "Semantic Web",
                                        "Workshop of Cross-Language Evaluation Forum",
                                        "International Semantic Web Conference",
                                        "ACM Transactions on Intelligent Systems and Technology",
                                        "TIST"
                                       ],
    
    "Human Computer Interaction": ["Computer Human Interaction",
                                   "CHI",
                                   "IEEE Transactions on Affective Computing",
                                   "Proceedings of the ACM on Interactive, Mobile, Wearable and Ubiquitous Technologies",
                                   "Proceedings of the ACM on Human-Computer Interaction",
                                   "International Journal of Human-Computer Studies",
                                   "ACM/IEEE International Conference on Human Robot Interaction",
                                   "ACM Conference on Computer-Supported Cooperative Work & Social Computing",
                                   "IEEE Transactions on Human-Machine Systems",
                                   "Behaviour & Information Technology",
                                   "ACM Symposium on User Interface Software and Technology",
                                   "International Journal of Human-Computer Interaction",
                                   "Designing Interactive Systems Conference",
                                   "International Conference on Intelligent User Interfaces",
                                   "IUI",
                                   "Universal Access in the Information Society",
                                   "Virtual Reality",
                                   "ACM Transactions on Computer-Human Interaction",
                                   "TOCHI",
                                   "IEEE Virtual Reality Conference",
                                   "International Journal of Interactive Mobile Technologies",
                                   "HCI International",
                                   "International Conference on Automotive User Interfaces and Interactive Vehicular Applications"
    ],
    
    "Theoretical Computer Science": ["ACM Symposium on Theory of Computing",
                                     "ACM SIAM Symposium on Discrete Algorithms",
                                     "IEEE Symposium on Foundations of Computer Science",
                                     "FOCS",
                                     "Journal of the ACM",
                                     "JACM",
                                     "SIAM Journal on Computing",
                                     "Theoretical Computer Science",
                                     "Conference on Innovations in Theoretical Computer Science",
                                     "ACM Transactions on Algorithms",
                                     "TALG",
                                     "IEEE Symposium on Logic in Computer Science",
                                     "International Colloquium on Automata, Languages and Programming",
                                     "ICALP",
                                     "Journal of Automated Reasoning",
                                     "Algorithmica",
                                     "Journal of Computer and System Sciences",
                                     "Fundamenta Informaticae",
                                     "Logical Methods in Computer Science",
                                     "Random Structures & Algorithms",
                                     "International Conference on Automated Deduction",
                                     "ACM Symposium on Parallelism in Algorithms and Architectures",
                                     "SPAA",
                                     "Combinatorica",
                                     "Mathematical Structures in Computer Science"
    ],
}

In [77]:
# target = "Computational Linguistics"
# target = "Computer Graphics"
# target = "Computer Networks & Wireless Communication"
# target = "Computer Vision & Pattern Recognition"
# target = "Computing Systems"
# target =  "Databases & Information Systems"
target =  "Human Computer Interaction"
target = "Theoretical Computer Science"

In [78]:
count = 0
for ven in venue_set:
    find = False
    for cand in venue_category[target]:
        if cand in str(ven):
            find = True
            break
    if find:
        count += 1
print(count)

663


In [79]:
d = pd.read_csv("paper_info.txt")
d

,paperid,venue,year
0,1,Linux Journal,1998
1,2,Integration the VLSI Journal,1984
2,3,International Journal of Parallel Programming,1984
3,4,International Journal of Parallel Programming,1984
4,5,International Journal of Parallel Programming,1984
...,...,...,...
2092140,2092352,ACM Transactions on Software Engineering and M...,2008
2092141,2092353,ACM Transactions on Software Engineering and M...,2008
2092142,2092354,ACM Transactions on Software Engineering and M...,2008
2092143,2092355,XRDS: Crossroads The ACM Magazine for Student...,2010


In [84]:
from tqdm import tqdm

filtered_paperid = []
paperid2categories = []
for i, row in tqdm(d.iterrows()):
    find = False
    for target in venue_category.keys():
        for cand in venue_category[target]:
            if cand in str(row["venue"]):
                find = True
                paperid2categories.append(target)
                break
        if find:
            break
    if find:
        filtered_paperid.append(row["paperid"])

2092145it [36:10, 964.10it/s] 


In [85]:
print(len(filtered_paperid))

199246


In [86]:
with open("sampled_paperid.txt","w") as f:
    for i, paperid in enumerate(filtered_paperid):
        f.write(str(paperid) + "\t" + str(paperid2categories[i]) + "\n")

In [3]:
%pwd

'/workspace/WithInHyperedgeNodeLabel/RankingAggregation'